In [1]:
#needed after restructuring of directory
import os
os.chdir('/Users/alexchandler/UFC_Prediction_2022')
os.getcwd()

'/Users/alexchandler/UFC_Prediction_2022'

In [ ]:
#This didn't end up working... future work to get it running
#this is all copied from https://pytorch.org/tutorials/advanced/neural_style_tutorial.html
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torchvision.models as models
import copy
from torchvision.utils import save_image
import torch
import torchvision
import random


In [ ]:
from multiprocess import Process

In [3]:
ufcfighterscrap =pd.read_csv('src/content/data/processed/fighter_stats.csv',sep=',',low_memory=False)
names = list(ufcfighterscrap['name'])

In [ ]:
def stylize(a, b):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cnn = models.vgg19(pretrained=True).features.to(device).eval()
    # desired size of the output image
    imsize = 512 if torch.cuda.is_available() else 256  # use small size if no gpu

    loader = transforms.Compose([
        transforms.Resize(imsize),  # scale imported image
        transforms.ToTensor()])  # transform it into a torch tensor


    def image_loader(image):
        # fake batch dimension required to fit network's input dimensions
        image = loader(image).unsqueeze(0)
        return image.to(device, torch.float)

    styleimage0=Image.open("src/content/images/escher1.jpeg")  
    styleimage1=Image.open("src/content/images/textile1.jpeg")
    styleimage2=Image.open("src/content/images/pixelart.jpeg")
    styleimage3=Image.open("src/content/images/joker.jpeg") 

    styleimage4=Image.open("src/content/images/conorpainting.jpeg")  
    styleimage5=Image.open("src/content/images/picasso.jpeg")  
    styleimage6=Image.open("src/content/images/paintedoblivion.webp")
    styleimage7=Image.open("src/content/images/starrynight.jpeg")
    styleimage8=Image.open("src/content/images/bear.jpeg")
    styleimage9=Image.open("src/content/images/god.jpeg")
    styleimage10=Image.open("src/content/images/vangogh.jpeg")  

    thresh={}
    thresh[0] = 560 #escher1 
    thresh[1] = 140 #textile1
    thresh[2]=900
    thresh[3]=120 
    thresh[4]=150 
    thresh[5]=150 
    thresh[6]=150 
    thresh[7]=150 
    thresh[8]=150 
    thresh[9]=100 
    thresh[10]=80


    styleimage0=styleimage0.resize((256,256))
    styleimage1=styleimage1.resize((256,256))
    styleimage2=styleimage2.resize((256,256))
    styleimage3=styleimage3.resize((256,256))
    styleimage4=styleimage4.resize((256,256))
    styleimage5=styleimage5.resize((256,256))
    styleimage6=styleimage6.resize((256,256))
    styleimage7=styleimage7.resize((256,256))
    styleimage8=styleimage8.resize((256,256))
    styleimage9=styleimage9.resize((256,256))
    styleimage10=styleimage10.resize((256,256))



    fighterimage=Image.open("src/content/images/2KhamzatChimaev.jpg")
    fighterimage=fighterimage.resize((256,256))

    style = [image_loader(styleimage0),image_loader(styleimage1),image_loader(styleimage2),
             image_loader(styleimage3), image_loader(styleimage4), image_loader(styleimage5), 
             image_loader(styleimage6), image_loader(styleimage7), image_loader(styleimage8),
            image_loader(styleimage9), image_loader(styleimage10)]

    content_img = image_loader(fighterimage)
    assert all(style[i].size() == content_img.size() for i in range(len(style))), \
        "we need to import style and content images of the same size"

    unloader = transforms.ToPILImage()  # reconvert into PIL image

    plt.ion()

    def imshow(tensor, title=None):
        image = tensor.cpu().clone()  # we clone the tensor to not do changes on it
        image = image.squeeze(0)      # remove the fake batch dimension
        image = unloader(image)
        plt.imshow(image)
        if title is not None:
            plt.title(title)
        plt.pause(0.001) # pause a bit so that plots are updated

    class ContentLoss(nn.Module):

        def __init__(self, target,):
            super(ContentLoss, self).__init__()
            # we 'detach' the target content from the tree used
            # to dynamically compute the gradient: this is a stated value,
            # not a variable. Otherwise the forward method of the criterion
            # will throw an error.
            self.target = target.detach()

        def forward(self, input):
            self.loss = F.mse_loss(input, self.target)
            return input

    def gram_matrix(input):
        a, b, c, d = input.size()  # a=batch size(=1)
        # b=number of feature maps
        # (c,d)=dimensions of a f. map (N=c*d)

        features = input.view(a * b, c * d)  # resise F_XL into \hat F_XL

        G = torch.mm(features, features.t())  # compute the gram product

        # we 'normalize' the values of the gram matrix
        # by dividing by the number of element in each feature maps.
        return G.div(a * b * c * d)

    class StyleLoss(nn.Module):

        def __init__(self, target_feature):
            super(StyleLoss, self).__init__()
            self.target = gram_matrix(target_feature).detach()

        def forward(self, input):
            G = gram_matrix(input)
            self.loss = F.mse_loss(G, self.target)
            return input

    cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
    cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

    # create a module to normalize input image so we can easily put it in a
    # nn.Sequential
    class Normalization(nn.Module):
        def __init__(self, mean, std):
            super(Normalization, self).__init__()
            # .view the mean and std to make them [C x 1 x 1] so that they can
            # directly work with image Tensor of shape [B x C x H x W].
            # B is batch size. C is number of channels. H is height and W is width.
            self.mean = torch.tensor(mean).view(-1, 1, 1)
            self.std = torch.tensor(std).view(-1, 1, 1)

        def forward(self, img):
            # normalize img
            return (img - self.mean) / self.std

    # desired depth layers to compute style/content losses :
    content_layers_default = ['conv_4']
    style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

    def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                                   style_img, content_img,
                                   content_layers=content_layers_default,
                                   style_layers=style_layers_default):
        # normalization module
        normalization = Normalization(normalization_mean, normalization_std).to(device)

        # just in order to have an iterable access to or list of content/syle
        # losses
        content_losses = []
        style_losses = []

        # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
        # to put in modules that are supposed to be activated sequentially
        model = nn.Sequential(normalization)

        i = 0  # increment every time we see a conv
        for layer in cnn.children():
            if isinstance(layer, nn.Conv2d):
                i += 1
                name = 'conv_{}'.format(i)
            elif isinstance(layer, nn.ReLU):
                name = 'relu_{}'.format(i)
                # The in-place version doesn't play very nicely with the ContentLoss
                # and StyleLoss we insert below. So we replace with out-of-place
                # ones here.
                layer = nn.ReLU(inplace=False)
            elif isinstance(layer, nn.MaxPool2d):
                name = 'pool_{}'.format(i)
            elif isinstance(layer, nn.BatchNorm2d):
                name = 'bn_{}'.format(i)
            else:
                raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

            model.add_module(name, layer)

            if name in content_layers:
                # add content loss:
                target = model(content_img).detach()
                content_loss = ContentLoss(target)
                model.add_module("content_loss_{}".format(i), content_loss)
                content_losses.append(content_loss)

            if name in style_layers:
                # add style loss:
                target_feature = model(style_img).detach()
                style_loss = StyleLoss(target_feature)
                model.add_module("style_loss_{}".format(i), style_loss)
                style_losses.append(style_loss)

        # now we trim off the layers after the last content and style losses
        for i in range(len(model) - 1, -1, -1):
            if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
                break

        model = model[:(i + 1)]

        return model, style_losses, content_losses

    input_img = content_img.clone()
    # if you want to use white noise instead uncomment the below line:
    # input_img = torch.randn(content_img.data.size(), device=device)

    # add the original input image to the figure:
    plt.figure()

    #for i in range(len(style)):
        #print('style '+str(i))
        #imshow(style[i], title='Style Image')

    def get_input_optimizer(input_img):
        # this line to show that input is a parameter that requires a gradient
        optimizer = optim.LBFGS([input_img])
        return optimizer

    def run_style_transfer(cnn, normalization_mean, normalization_std,
                           content_img, style_img, input_img, num_steps=300,
                           style_weight=1000000, content_weight=1):
        """Run the style transfer."""
        model, style_losses, content_losses = get_style_model_and_losses(cnn,
            normalization_mean, normalization_std, style_img, content_img)
        # We want to optimize the input and not the model parameters so we
        # update all the requires_grad fields accordingly
        input_img.requires_grad_(True)
        model.requires_grad_(False)
        optimizer = get_input_optimizer(input_img)
        run = [0]
        while run[0] <= num_steps:
            def closure():
                # correct the values of updated input image
                with torch.no_grad():
                    input_img.clamp_(0, 1)
                optimizer.zero_grad()
                model(input_img)
                style_score = 0
                content_score = 0
                for sl in style_losses:
                    style_score += sl.loss
                for cl in content_losses:
                    content_score += cl.loss
                style_score *= style_weight
                content_score *= content_weight
                loss = style_score + content_score
                loss.backward()
                run[0] += 1
                if run[0] ==50:
                    print('Style Loss : {:4f}'.format(style_score.item()))
                return style_score + content_score
            optimizer.step(closure)
        # a last correction...
        with torch.no_grad():
            input_img.clamp_(0, 1)
        return input_img

    def run_style_transfer_adaptive(cnn, normalization_mean, normalization_std,
                           content_img, styleindex, input_img, num_steps=300,
                           style_weight=1000000, content_weight=1):
        style_img=style[styleindex]
        firstLoss={0:0}
        """Run the style transfer."""
        model, style_losses, content_losses = get_style_model_and_losses(cnn,
            normalization_mean, normalization_std, style_img, content_img)
        # We want to optimize the input and not the model parameters so we
        # update all the requires_grad fields accordingly
        input_img.requires_grad_(True)
        model.requires_grad_(False)
        optimizer = get_input_optimizer(input_img)
        run = [0]
        while run[0] <= num_steps and firstLoss[0]==0:
            def closure():
                # correct the values of updated input image
                with torch.no_grad():
                    input_img.clamp_(0, 1)
                optimizer.zero_grad()
                model(input_img)
                style_score = 0
                content_score = 0
                for sl in style_losses:
                    style_score += sl.loss
                for cl in content_losses:
                    content_score += cl.loss
                style_score *= style_weight
                content_score *= content_weight
                loss = style_score + content_score
                loss.backward()
                run[0] += 1
                if run[0] % 50 == 0:
                    if style_score.item()>thresh[styleindex]:
                        firstLoss[0]=style_score.item()
                return style_score + content_score
            optimizer.step(closure)
        # a last correction...
        with torch.no_grad():
            input_img.clamp_(0, 1)
        if firstLoss[0]==0:
            return input_img
        else:
            return 'empty'
    i=a-1
    for name in names[a:b]:
        i+=1
        for j in range(1,5):
            try:
                name = name.replace(" ","")
                fighterimage=Image.open("src/content/images/"+str(j)+name+".jpg")
                fighterimage=fighterimage.resize((256,256))
                content_img = image_loader(fighterimage)
                styleindex=random.randint(1,len(style))-1
                output = 'empty'
                k=0
                while output =='empty':
                    if k>len(style):
                        print(str(j)+' couldnt find a good style for '+str(j)+name)
                        break
                    k+=1
                    input_img = content_img.clone()
                    styleindex=(styleindex+1)%len(style)
                    output = run_style_transfer_adaptive(cnn, cnn_normalization_mean, cnn_normalization_std,
                                       content_img, styleindex, input_img, num_steps=300,
                                       style_weight=1000000, content_weight=1)
                img1 = output
                save_image(img1, "src/content/images2/"+str(j)+name+".jpg")
                print(str(i)+' saving: '+"src/content/images2/"+str(j)+name+".jpg"+' in style '+str(styleindex))
            except:
                print('exception')
                continue

In [ ]:
#remaining tasks 370-1806 without 600-650
# construct a different process for each function
processes = [Process(target=stylize, args=(200,203)),
             #Process(target=stylize, args=(370,514)),
             #Process(target=stylize, args=(514,601)),
             #600-650 done on work desktop so we can skip this part
             #Process(target=stylize, args=(650,802)),
             #Process(target=stylize, args=(802,946)),
             #Process(target=stylize, args=(946,1090)),
             #Process(target=stylize, args=(1090,1234)),
             #Process(target=stylize, args=(1234,1378)),
             #Process(target=stylize, args=(1378,1522)),
             #Process(target=stylize, args=(1522,1666)),
             #Process(target=stylize, args=(1666,1806)),
            ]

# kick them off 
for process in processes:
    process.start()

# now wait for them to finish
for process in processes:
    process.join()

In [ ]:
#started run_in_parallel at 8:32 pm friday april 8

In [ ]:
#this is the original example that inspired this notebook
#https://stackoverflow.com/questions/47368904/control-python-code-to-run-on-different-core
from multiprocess import Process

def add(a, b):
    total = 0
    for a1, b1 in zip(a, b):
        total = a1 + b1
    return total

def sub(s, t):
    total = 0
    for a1, b1 in zip(s, t):
        total = a1 - b1
    return total

def mult(y, x):
    total = 0
    for a1, b1 in zip(y, x):
        total = a1 * b1
    return total

if __name__ == "__main__":
    # construct a different process for each function
    max_size = 1000000000
    processes = [Process(target=add, args=(range(1, max_size), range(1, max_size))),
                 Process(target=sub, args=(range(1, max_size), range(1, max_size))),
                 Process(target=mult, args=(range(1, max_size), range(1, max_size)))]

    # kick them off 
    for process in processes:
        process.start()

    # now wait for them to finish
    for process in processes:
        process.join()

In [ ]:
#try without the __main__ thing
#this is the original example that inspired this notebook
#https://stackoverflow.com/questions/47368904/control-python-code-to-run-on-different-core
from multiprocess import Process

def add(a, b):
    total = 0
    for a1, b1 in zip(a, b):
        total = a1 + b1
    return total

def sub(s, t):
    total = 0
    for a1, b1 in zip(s, t):
        total = a1 - b1
    return total

def mult(y, x):
    total = 0
    for a1, b1 in zip(y, x):
        total = a1 * b1
    return total

# construct a different process for each function
max_size = 1000000000
processes = [Process(target=add, args=(range(1, max_size), range(1, max_size))),
             Process(target=sub, args=(range(1, max_size), range(1, max_size))),
             Process(target=mult, args=(range(1, max_size), range(1, max_size)))]

# kick them off 
for process in processes:
    process.start()

# now wait for them to finish
for process in processes:
    process.join()

In [12]:
#!/usr/bin/python
from multiprocess import Process
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torchvision.models as models
import copy
from torchvision.utils import save_image
import torch
import torchvision
import random
import time
def fun(val):
    print(f'starting fun with {val} s')
    time.sleep(val)
    print(f'finishing fun with {val} s')
    
def single_stylize(a):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cnn = models.vgg19(pretrained=True).features.to(device).eval()
    ufcfighterscrap =pd.read_csv('src/content/data/processed/fighter_stats.csv',sep=',',low_memory=False)
    names = list(ufcfighterscrap['name'])
    # desired size of the output image
    imsize = 512 if torch.cuda.is_available() else 256  # use small size if no gpu

    loader = transforms.Compose([
        transforms.Resize(imsize),  # scale imported image
        transforms.ToTensor()])  # transform it into a torch tensor


    def image_loader(image):
        # fake batch dimension required to fit network's input dimensions
        image = loader(image).unsqueeze(0)
        return image.to(device, torch.float)

    styleimage0=Image.open("src/content/images/escher1.jpeg")  
    styleimage1=Image.open("src/content/images/textile1.jpeg")
    styleimage2=Image.open("src/content/images/pixelart.jpeg")
    styleimage3=Image.open("src/content/images/joker.jpeg") 

    styleimage4=Image.open("src/content/images/conorpainting.jpeg")  
    styleimage5=Image.open("src/content/images/picasso.jpeg")  
    styleimage6=Image.open("src/content/images/paintedoblivion.webp")
    styleimage7=Image.open("src/content/images/starrynight.jpeg")
    styleimage8=Image.open("src/content/images/bear.jpeg")
    styleimage9=Image.open("src/content/images/god.jpeg")
    styleimage10=Image.open("src/content/images/vangogh.jpeg")  

    thresh={}
    thresh[0] = 560 #escher1 
    thresh[1] = 140 #textile1
    thresh[2]=900
    thresh[3]=120 
    thresh[4]=150 
    thresh[5]=150 
    thresh[6]=150 
    thresh[7]=150 
    thresh[8]=150 
    thresh[9]=100 
    thresh[10]=80


    styleimage0=styleimage0.resize((256,256))
    styleimage1=styleimage1.resize((256,256))
    styleimage2=styleimage2.resize((256,256))
    styleimage3=styleimage3.resize((256,256))
    styleimage4=styleimage4.resize((256,256))
    styleimage5=styleimage5.resize((256,256))
    styleimage6=styleimage6.resize((256,256))
    styleimage7=styleimage7.resize((256,256))
    styleimage8=styleimage8.resize((256,256))
    styleimage9=styleimage9.resize((256,256))
    styleimage10=styleimage10.resize((256,256))



    fighterimage=Image.open("src/content/images/2KhamzatChimaev.jpg")
    fighterimage=fighterimage.resize((256,256))

    style = [image_loader(styleimage0),image_loader(styleimage1),image_loader(styleimage2),
             image_loader(styleimage3), image_loader(styleimage4), image_loader(styleimage5), 
             image_loader(styleimage6), image_loader(styleimage7), image_loader(styleimage8),
            image_loader(styleimage9), image_loader(styleimage10)]

    content_img = image_loader(fighterimage)
    assert all(style[i].size() == content_img.size() for i in range(len(style))), \
        "we need to import style and content images of the same size"

    unloader = transforms.ToPILImage()  # reconvert into PIL image

    plt.ion()

    def imshow(tensor, title=None):
        image = tensor.cpu().clone()  # we clone the tensor to not do changes on it
        image = image.squeeze(0)      # remove the fake batch dimension
        image = unloader(image)
        plt.imshow(image)
        if title is not None:
            plt.title(title)
        plt.pause(0.001) # pause a bit so that plots are updated

    class ContentLoss(nn.Module):

        def __init__(self, target,):
            super(ContentLoss, self).__init__()
            # we 'detach' the target content from the tree used
            # to dynamically compute the gradient: this is a stated value,
            # not a variable. Otherwise the forward method of the criterion
            # will throw an error.
            self.target = target.detach()

        def forward(self, input):
            self.loss = F.mse_loss(input, self.target)
            return input

    print('checkpoint1')
    def gram_matrix(input):
        a, b, c, d = input.size()  # a=batch size(=1)
        # b=number of feature maps
        # (c,d)=dimensions of a f. map (N=c*d)

        features = input.view(a * b, c * d)  # resise F_XL into \hat F_XL

        G = torch.mm(features, features.t())  # compute the gram product

        # we 'normalize' the values of the gram matrix
        # by dividing by the number of element in each feature maps.
        return G.div(a * b * c * d)

    class StyleLoss(nn.Module):

        def __init__(self, target_feature):
            super(StyleLoss, self).__init__()
            self.target = gram_matrix(target_feature).detach()

        def forward(self, input):
            G = gram_matrix(input)
            self.loss = F.mse_loss(G, self.target)
            return input        
        
    cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).clone().detach().to(device)
    cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).clone().detach().to(device)

    # create a module to normalize input image so we can easily put it in a
    # nn.Sequential
    class Normalization(nn.Module):
        def __init__(self, mean, std):
            super(Normalization, self).__init__()
            # .view the mean and std to make them [C x 1 x 1] so that they can
            # directly work with image Tensor of shape [B x C x H x W].
            # B is batch size. C is number of channels. H is height and W is width.
            self.mean = (mean).clone().detach().view(-1, 1, 1)
            self.std = (std).clone().detach().view(-1, 1, 1)

        def forward(self, img):
            # normalize img
            return (img - self.mean) / self.std

    # desired depth layers to compute style/content losses :
    content_layers_default = ['conv_4']
    style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']
    print('checkpoint2')
    def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                                   style_img, content_img,
                                   content_layers=content_layers_default,
                                   style_layers=style_layers_default):
        print('made it into get_style_model_...')
        # normalization module
        normalization = Normalization(normalization_mean, normalization_std).to(device)
        print('normalized')
        # just in order to have an iterable access to or list of content/syle
        # losses
        content_losses = []
        style_losses = []

        # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
        # to put in modules that are supposed to be activated sequentially
        model = nn.Sequential(normalization)
        print('called model')
        i = 0  # increment every time we see a conv
        for layer in cnn.children():
            print('in the for loop layer')
            if isinstance(layer, nn.Conv2d):
                print('in if')
                i += 1
                name = 'conv_{}'.format(i)
            elif isinstance(layer, nn.ReLU):
                print('in elif')
                name = 'relu_{}'.format(i)
                # The in-place version doesn't play very nicely with the ContentLoss
                # and StyleLoss we insert below. So we replace with out-of-place
                # ones here.
                layer = nn.ReLU(inplace=False)
            elif isinstance(layer, nn.MaxPool2d):
                print('in 2nd elif')
                name = 'pool_{}'.format(i)
            elif isinstance(layer, nn.BatchNorm2d):
                print('in 3rd elif')
                name = 'bn_{}'.format(i)
            else:
                print('in else')
                raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))
            print('before add module')
            model.add_module(name, layer)
            print('after add module')
            if name in content_layers:
                # add content loss:
                print('in if name')
                target = model(content_img).detach()
                content_loss = ContentLoss(target)
                model.add_module("content_loss_{}".format(i), content_loss)
                content_losses.append(content_loss)

            if name in style_layers:
                print('in if name style layers')
                # add style loss:
                #GETS HUNG HERE... on model(style_img)
                target_feature = model(style_img).detach()
                print('target_feature')
                style_loss = StyleLoss(target_feature)
                print('style_loss')
                model.add_module("style_loss_{}".format(i), style_loss)
                print('model.add')
                style_losses.append(style_loss)
                print('style losses append')
        print('made it past this for loop')
        # now we trim off the layers after the last content and style losses
        for i in range(len(model) - 1, -1, -1):
            if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
                break

        model = model[:(i + 1)]

        return model, style_losses, content_losses
    print('checkpoint3')
    input_img = content_img.clone()
    # if you want to use white noise instead uncomment the below line:
    # input_img = torch.randn(content_img.data.size(), device=device)

    # add the original input image to the figure:
    plt.figure()

    #for i in range(len(style)):
        #print('style '+str(i))
        #imshow(style[i], title='Style Image')

    def get_input_optimizer(input_img):
        # this line to show that input is a parameter that requires a gradient
        optimizer = optim.LBFGS([input_img])
        return optimizer
    print('checkpoint4')
    

    def run_style_transfer_adaptive(cnn, normalization_mean, normalization_std,
                           content_img, styleindex, input_img, num_steps=300,
                           style_weight=1000000, content_weight=1):
        print('made it into run_style_transfer_adaptive')
        style_img=style[styleindex]
        firstLoss={0:0}
        """Run the style transfer."""
        print('got to this place before model, style')
        model, style_losses, content_losses = get_style_model_and_losses(cnn,
            normalization_mean, normalization_std, style_img, content_img)
        # We want to optimize the input and not the model parameters so we
        # update all the requires_grad fields accordingly
        print('got to this place before input_img')
        input_img.requires_grad_(True)
        model.requires_grad_(False)
        optimizer = get_input_optimizer(input_img)
        print('after optimizer')
        run = [0]
        while run[0] <= num_steps and firstLoss[0]==0:
            def closure():
                # correct the values of updated input image
                with torch.no_grad():
                    input_img.clamp_(0, 1)
                optimizer.zero_grad()
                model(input_img)
                style_score = 0
                content_score = 0
                for sl in style_losses:
                    style_score += sl.loss
                for cl in content_losses:
                    content_score += cl.loss
                style_score *= style_weight
                content_score *= content_weight
                loss = style_score + content_score
                loss.backward()
                run[0] += 1
                if run[0] % 50 == 0:
                    if style_score.item()>thresh[styleindex]:
                        firstLoss[0]=style_score.item()
                return style_score + content_score
            optimizer.step(closure)
        # a last correction...
        with torch.no_grad():
            input_img.clamp_(0, 1)
        if firstLoss[0]==0:
            return input_img
        else:
            return 'empty'
    print('checkpoint5')
    i=a-1
    for name in names[a:a+3]:
        i+=1
        for j in range(1,5):
            try:
                name = name.replace(" ","")
                print('checkpoint6')
                fighterimage=Image.open("src/content/images/"+str(j)+name+".jpg")
                print('checkpoint7')
                fighterimage=fighterimage.resize((256,256))
                content_img = image_loader(fighterimage)
                styleindex=random.randint(1,len(style))-1
                output = 'empty'
                print('checkpoint8')
                k=0
                while output =='empty':
                    if k>len(style):
                        print(str(j)+' couldnt find a good style for '+str(j)+name)
                        break
                    k+=1
                    input_img = content_img.clone()
                    styleindex=(styleindex+1)%len(style)
                    print('checkpoint9')
                    output = run_style_transfer_adaptive(cnn, cnn_normalization_mean, cnn_normalization_std,
                                       content_img, styleindex, input_img, num_steps=300,
                                       style_weight=1000000, content_weight=1)
                    print('checkpoint10')
                img1 = output
                print('checkpoint11')
                save_image(img1, "src/content/images2/"+str(j)+name+".jpg")
                print(str(i)+' saving: '+"src/content/images2/"+str(j)+name+".jpg"+' in style '+str(styleindex))
            except:
                print('exception')
                continue
    print('checkpoint6')
    
def main():
    p1 = Process(target=single_stylize, args=(3, ))
    p1.start()
    p2 = Process(target=fun, args=(2, ))
    p2.start()
    p3 = Process(target=fun, args=(1, ))
    p3.start()
    p1.join()
    p2.join()
    p3.join()
    print('finished main')
if __name__ == '__main__':
    main()

starting fun with 2 s
starting fun with 1 s
checkpoint1
checkpoint2
checkpoint3
checkpoint4
checkpoint5
checkpoint6
checkpoint7
checkpoint8
checkpoint9
made it into run_style_transfer_adaptive
got to this place before model, style
made it into get_style_model_...
normalized
called model
in the for loop layer
in if
before add module
after add module
in if name style layers
finishing fun with 1 s
finishing fun with 2 s
finished main
